In [117]:
import plotly.express as px
import pandas as pd
import requests

url = "https://data.giss.nasa.gov/gistemp/tabledata_v4/GLB.Ts+dSST.txt"
response = requests.get(url)

if response.status_code == 200:
    lines = response.text.split('\n')
    for i, line in enumerate(lines):
        if 'Year' in line:
            start_line = i
            break
    data = lines[start_line:]
    df = pd.DataFrame([sub.split() for sub in data])
    df.columns = df.iloc[0]
    df = df.drop(0)
    df = df[['Year', 'J-D']]
    df = df.dropna()

    # Drop duplicate 'Year' columns
    df = df.loc[:, ~df.columns.duplicated()]

    # Filter out rows where 'Year' column has the value 'Year'
    df = df[df['Year'] != 'Year']


    # Convert 'Year' column to numeric
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

else:
    print("Failed to retrieve data from the URL.")

df['J-D'] = pd.to_numeric(df['J-D'], errors='coerce')
df['J-D'] = df['J-D'] / 100
df.rename(columns={'J-D': 'Average Temperature (°C)'}, inplace=True)

# Remove the row with the maximum year
max_year = df['Year'].max()
df = df[df['Year'] != max_year]

# Define the custom color scale
color_scale = [(0, '#498aea'), (0.5, 'white'), (1, '#dd1d1f')]

# Create a new DataFrame for plotting
plot_df = df.copy()
plot_df['y'] = 1  # Add a y column with all values set to 1

# Create the bar chart with custom color scale
fig = px.bar(plot_df, x='Year', y='y',  # Use the new y column
             color='Average Temperature (°C)',  # Conditional formatting based on 'Average Temperature (°C)'
             color_continuous_scale=color_scale,  # Custom color scale
             labels={'y': 'Value'},  # Customize y-axis label
             hover_data={'y': False})  # Remove y value from tooltip

# Customize the layout
fig.update_layout(
    width=1200,  # Set the width of the figure
    height=800,  # Set the height of the figure
    plot_bgcolor='black',  # Black background
    paper_bgcolor='black',  # Black paper background
    xaxis_title='',  # Remove x-axis title
    yaxis_title='',  # Remove y-axis title
    showlegend=False,  # Remove legend
    title='',  # Remove visual title
    coloraxis_showscale=False,  # Remove color scale legend
    xaxis=dict(
        tickfont=dict(color='#666666'),  # Change x-axis tick labels color
        showgrid=False  # Remove x-axis grid lines
    ),
    yaxis=dict(
        tickfont=dict(color='#666666'),  # Change y-axis tick labels color
        showgrid=False,  # Remove y-axis grid lines
        showticklabels=False  # Remove y-axis values
    )  # Adjust margins to zoom in
)

# Customize hover template to bold values, set font, and increase font size
fig.update_traces(
    hovertemplate='<span style="font-family: \'Segoe UI\'; font-size: 16px;"><b>Year</b>: %{x}<br><b>Average Temperature (°C)</b>: %{marker.color:.2f}</span>'
)

# Display the bar chart
fig.show()
